In [2]:
import pandas as pd
import numpy as np
from math import sin, cos, sqrt, atan2, radians
from tqdm import tqdm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
# magic command to display matplotlib plots inline within the ipython notebook
%matplotlib inline

# import necessary modules
import pandas as pd, numpy as np, matplotlib.pyplot as plt, time
from sklearn.cluster import DBSCAN
from sklearn import metrics
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
import catboost as cb

In [12]:
import re

In [3]:
def rmse(y_true, y_pred):
    return sqrt(mean_squared_error(y_true, y_pred))

In [4]:
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)

In [5]:
train['isTrain'] = True
test['isTrain'] = False

In [6]:
X = train.append(test)

In [7]:
X.isnull().sum()

address           0
address_rus     420
atm_group         0
id                0
isTrain           0
lat             420
long            420
target         2504
dtype: int64

# Геокодирование + заполнение пропусков

In [9]:
#https://geocode-maps.yandex.ru/1.x/?geocode=Moscow+Lva+Tolstogo+Street+16&lang=en_US
import requests
import json
YANDEX_URL = "https://geocode-maps.yandex.ru/1.x"
STATIC = "/?format=json&lang=en_US&geocode="
def geocode(address):
        response = requests.get(YANDEX_URL + STATIC + address)
        parsed_dict = json.loads(response.content)
        try:
            pos =  parsed_dict['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos']
            lat, lon = [float(item) for item in pos.split()]
            return lat, lon
        except:
            return None, None

In [13]:
def update_address(address):
    if re.match(r'.+\sG\s+', address):
        return re.sub('\sG\s+', '', address)
    else:
        return address

In [14]:
X.address = X.address.apply(lambda x: update_address(x))

In [15]:
X[['lat','long']] = X.apply(lambda row: geocode(row['address']) if (np.isnan(row['lat'])|np.isnan(row['long'])) else (row['lat'],row['long']), axis = 1).apply(pd.Series)

In [19]:
X.isnull().sum()

address           0
address_rus     420
atm_group         0
id                0
isTrain           0
lat              76
long             76
target         2504
dtype: int64

In [20]:
# остатки заполняла руками - 76 штук (надо попробовать selenium)

In [21]:
X[X.lat.isnull()==True]

,address,address_rus,atm_group,id,isTrain,lat,long,target
40,PER. BAZARNY 4 SPASSK-DALNY,NaN,496.5,4510.0,True,NaN,NaN,0.039191
150,ABB 6B NAB-CHELNINSKIJ NAB.CHELNY,NaN,1022.0,347.0,True,NaN,NaN,-0.145001
475,"KR.G. ROSSIJSKAJA,6 OMSK",NaN,3185.5,2746.0,True,NaN,NaN,-0.122659
506,"D. 23, KORP. 1, UL. MATRO BIYSK",NaN,5478.0,8436.0,True,NaN,NaN,-0.137240
566,"D. 91, PR-KT OBVODNYI KAN ARKHANGELSK",NaN,5478.0,5021.0,True,NaN,NaN,-0.069187
700,"D. 47A, PR-KT KRASNOARMEI BARNAUL",NaN,5478.0,5980.0,True,NaN,NaN,0.015838
751,"D. 2, KORP. 2, B-R SVOBOD SURGUT",NaN,5478.0,6404.0,True,NaN,NaN,-0.028232
780,"D. 262, UL. MAKSIMA GOR'K NIZHNIY NOVG",NaN,5478.0,6576.0,True,NaN,NaN,-0.073875
888,"D. 116/2, UL. KRASNOARMEI KRASNODAR",NaN,5478.0,7303.0,True,NaN,NaN,-0.020320
893,"D. 19/1, MKR. UBILEINYI IRKUTSK",NaN,5478.0,7343.0,True,NaN,NaN,-0.079420


In [ ]:
row = 2259
lat, longg = 55.682336, 52.295119

In [ ]:
X.at[row,'lat'] = lat
X.at[row,'long'] = longg

In [18]:
#X.to_csv('X_without_nulls.csv')